# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902009


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:16,  2.74s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:07,  2.49s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:47,  1.82s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:33,  1.33s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.29it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.77it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  3.77it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.97it/s]

not-logged-in-8c0c44faf754d344b18a     0.000811
equidad1                               0.089438
not-logged-in-1c83584f81d2151be85a     0.000567
klmasters                              0.000626
not-logged-in-27a50043cebc294af948     0.000564
Jes8x                                  0.000932
xperry9702                             0.001702
not-logged-in-d92e575bbfb01a0095ac     0.001162
not-logged-in-8c0c44faf754d344b18a     0.050536
Drew_Farnsworth                        0.000827
astro_lab_ncmns                        0.001588
ErickRico                              0.003613
hopefuji                               0.002322
pangeli5                               0.000643
nicholaschua                           0.001355
not-logged-in-687931fdaee12bdb17bd    94.618499
Lavadude                               0.017461
not-logged-in-a4bed12d9111c0c34e9e     0.006435
w7250369                               0.003185
deboi                                  0.004302
not-logged-in-2a9911e10449622ed772     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())